# Chess AI Toolkit Demo

This notebook demonstrates the key features of the Chess AI toolkit for feature explainability analysis and interactive chess gameplay.

## What is Chess AI?

A comprehensive chess AI toolkit featuring:

1. **Feature Explainability Audit**: Evaluate how well chess features explain Stockfish's evaluations using machine learning
2. **Explainable Chess Engine**: Play against Stockfish with move explanations and learning feedback  
3. **Advanced Positional Metrics**: Sophisticated chess position analysis including passed pawn momentum, king safety, and piece activity
4. **Kendall Tau Correlation**: Statistical analysis of move ranking correlations

**Key Benefits:**
- Understand why chess engines make specific moves
- Learn chess through interactive gameplay with explanations
- Analyze chess positions using advanced metrics
- Research chess AI explainability

## Setup and Installation

In [ ]:
! [ ! -d "chess" ] && git clone https://github.com/bangyen/chess.git
! cd chess && pip install -e .

print("Setup complete!")

## Imports and Configuration

In [ ]:
import os

os.chdir('./chess')
print(f"Current working directory: {os.getcwd()}")

In [ ]:
import chess
import numpy as np

from src.chess_ai import baseline_extract_features
from src.chess_ai.explainable_engine import ExplainableChessEngine
from src.chess_ai.metrics.positional import passed_pawn_momentum_snapshot

# Set random seed for reproducibility
np.random.seed(42)

print("Chess AI toolkit imported successfully!")

## 1. Feature Explainability Audit


In [ ]:
# Create sample chess positions for analysis
def create_sample_positions():
    """Create a variety of chess positions for feature analysis."""
    positions = []

    # Starting position
    board = chess.Board()
    positions.append(board.copy())

    # After e4 e5
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")
    positions.append(board.copy())

    # After e4 e5 Nf3 Nc6
    board.push_san("Nf3")
    board.push_san("Nc6")
    positions.append(board.copy())

    # Tactical position with hanging piece
    board = chess.Board("r1bqkb1r/pppp1ppp/2n2n2/4p3/2B1P3/3P1N2/PPP2PPP/RNBQK2R w KQkq - 4 4")
    positions.append(board.copy())

    # Endgame position
    board = chess.Board("8/8/8/8/8/8/4K3/4k3 w - - 0 1")
    positions.append(board.copy())

    return positions

# Generate sample positions
sample_positions = create_sample_positions()
print(f"Created {len(sample_positions)} sample positions for analysis")

In [ ]:
# Extract features from sample positions
print("Extracting features from sample positions...")
for i, board in enumerate(sample_positions):
    features = baseline_extract_features(board)
    print(f"\nPosition {i+1}:")
    print(f"  Material difference: {features['material_diff']:.1f}")
    print(f"  Mobility (us/them): {features['mobility_us']:.0f}/{features['mobility_them']:.0f}")
    print(f"  King safety (us/them): {features['king_safety_us']:.1f}/{features['king_safety_them']:.1f}")
    print(f"  Center control (us/them): {features['center_control_us']:.0f}/{features['center_control_them']:.0f}")
    print(f"  Passed pawns (us/them): {features['passed_us']:.0f}/{features['passed_them']:.0f}")

## 2. Explainable Chess Engine Demo

In [ ]:
# Demo the explainable chess engine
def demo_move_analysis():
    """Demonstrate move analysis and explanations."""
    print("Chess Move Analysis Demo")
    print("=" * 40)

    # Create a sample position
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")
    board.push_san("Nf3")
    board.push_san("Nc6")

    print("Sample position after 1.e4 e5 2.Nf3 Nc6:")
    print(board)

    # Analyze some moves
    moves_to_analyze = ["Bc4", "Bb5", "d3", "Nc3"]

    # Use the engine with proper context management
    with ExplainableChessEngine() as engine:
        for move_str in moves_to_analyze:
            try:
                move = board.parse_san(move_str)
                if move in board.legal_moves:
                    print(f"\nAnalyzing move: {move_str}")

                    explanation = engine.explain_move_with_board(move, board)

                    print(f"Move {move_str}:")
                    if explanation.reasons:
                        for reason in explanation.reasons[:3]:  # Show top 3 reasons
                            print(f"  • {reason[2]}")
                    else:
                        print(f"  • {explanation.overall_explanation}")
                else:
                    print(f"\n{move_str} is not a legal move in this position")
            except Exception as e:
                print(f"Error analyzing {move_str}: {e}")

# Run the demo
demo_move_analysis()


## 3. Positional Metrics Demo

In [ ]:
# Demonstrate advanced positional metrics
def demo_positional_metrics():
    """Show advanced chess positional analysis."""
    print("Advanced Positional Metrics Demo")
    print("=" * 40)

    # Create a position with passed pawns
    board = chess.Board("8/8/8/8/4P3/8/8/8 w - - 0 1")  # White pawn on e4

    print("Position with passed pawn:")
    print(board)

    # Analyze passed pawn momentum
    momentum_white = passed_pawn_momentum_snapshot(board, chess.WHITE)
    momentum_black = passed_pawn_momentum_snapshot(board, chess.BLACK)

    print("\nPassed Pawn Analysis:")
    print(f"White: {momentum_white['pp_count']} passed pawns")
    print(f"Black: {momentum_black['pp_count']} passed pawns")

    # Show feature extraction for this position
    features = baseline_extract_features(board)
    print("\nKey Features:")
    print(f"Material difference: {features['material_diff']:.1f}")
    print(f"Mobility: {features['mobility_us']:.0f} vs {features['mobility_them']:.0f}")
    print(f"King safety: {features['king_safety_us']:.1f} vs {features['king_safety_them']:.1f}")
    print(f"Center control: {features['center_control_us']:.0f} vs {features['center_control_them']:.0f}")

# Run the demo
demo_positional_metrics()


## 4. Interactive Chess Game

In [ ]:
# Interactive chess game (simplified demo)
def interactive_demo():
    """Simplified interactive chess demo."""
    print("Interactive Chess Demo")
    print("=" * 30)
    print("Commands: 'best' for best move, 'quit' to exit")
    print("Note: This is a simplified demo. For full interactive play, use:")
    print("chess-ai play --strength beginner")
    print()

    # Create a simple position
    board = chess.Board()
    board.push_san("e4")
    board.push_san("e5")

    print("Position after 1.e4 e5:")
    print(board)

    # Show some legal moves
    legal_moves = list(board.legal_moves)
    print(f"\nLegal moves: {', '.join([board.san(move) for move in legal_moves[:8]])}")

    # Demonstrate move analysis
    print("\nAnalyzing some moves:")
    with ExplainableChessEngine() as engine:
        for move_str in ["Nf3", "Bc4", "Nc3"]:
            try:
                move = board.parse_san(move_str)
                if move in legal_moves:
                    explanation = engine.explain_move_with_board(move, board)
                    print(f"\n{move_str}:")
                    if explanation.reasons:
                        for reason in explanation.reasons[:2]:
                            print(f"  • {reason[2]}")
                    else:
                        print(f"  • {explanation.overall_explanation}")
            except Exception as e:
                print(f"Error analyzing {move_str}: {e}")

# Run the demo
interactive_demo()


## Key Takeaways

This demo shows the Chess AI toolkit's capabilities:

1. **Feature Extraction**: Analyze chess positions using comprehensive feature sets including material, mobility, king safety, and tactical elements
2. **Move Analysis**: Get explanations for why moves are good or bad with specific reasoning
3. **Positional Metrics**: Advanced analysis of passed pawns, king safety, and piece activity
4. **Interactive Learning**: Play chess with real-time move explanations and feedback

### Next Steps

- **Full Feature Audit**: `chess-ai audit --baseline_features --positions 100`
- **Interactive Play**: `chess-ai play --strength intermediate`  
- **Custom Features**: Create your own feature extraction functions
- **Research**: Use the toolkit for chess AI explainability research

### Installation & Usage

```bash
# Install the toolkit
pip install chess-ai

# Run feature audit
chess-ai audit --positions 50

# Play interactive chess
chess-ai play --strength beginner
```

For more information, visit the [GitHub repository](https://github.com/bangyen/chess).
